In [1]:
# import packages and data
import numpy as np
import pandas as pd
import os
import seaborn as sns
import Bio
import matplotlib.pyplot as plt
import ast
import scipy.stats as stats
from scipy.optimize import curve_fit
import pickle
import numpy as np
from scipy.stats import wilcoxon

path: str = './data/'
prokaryotes_all: pd.DataFrame = pd.read_csv(os.path.join(path, "prokaryotes_all.csv"), dtype=str)
prokaryotes_values: pd.DataFrame = pd.read_csv(os.path.join(path, "prokaryotes_funct.csv"), dtype=str)
prokaryotes_lysate: pd.DataFrame = pd.read_csv(os.path.join(path, "prokaryotes_322columns.csv"))

prokaryotes_values['a_point'] = prokaryotes_values['a_point'].astype('float64')
prokaryotes_values['b_point'] = prokaryotes_values['b_point'].astype('float64')
prokaryotes_values['plateau'] = prokaryotes_values['plateau'].astype('float64')
prokaryotes_values['meltPoint'] = prokaryotes_values['meltPoint'].astype('float64')

mutated_proteins: pd.DataFrame = pd.read_csv(os.path.join(path, "Essential_proteins_prediction_merged.csv"), dtype=str)
mutated_proteins['meltPoint_x'] = mutated_proteins['meltPoint_x'].astype('float64')
mutated_proteins['WT_prediction'] = mutated_proteins['WT_prediction'].astype('float64')
mutated_proteins['Mut_prediction'] = mutated_proteins['Mut_prediction'].astype('float64')
mutated_proteins['WT_diff'] = mutated_proteins['WT_diff'].astype('float64')
mutated_proteins['Mut_diff'] = mutated_proteins['Mut_diff'].astype('float64')
mutated_proteins['error_WTpred'] = mutated_proteins['WT_prediction'] - mutated_proteins['meltPoint_x']

Solving of curve equation for f(T90) = 0.1 and value is stored as a new column in the prokaryotes_lysate data frame. 

In [ ]:
def curve_equation(x, a, b, plateau, target):
    rel_amount = ((1-plateau)/(1+(np.power(np.e, (-a/x)+b)))) + plateau
    return rel_amount - target
from scipy.optimize import fsolve
prokaryotes_values['T90'] = pd.Series(prokaryotes_values.index).apply(lambda i: fsolve(curve_equation, prokaryotes_values.loc[i, 'meltPoint'], args=(prokaryotes_values.loc[i,'a_point'], prokaryotes_values.loc[i,'b_point'], prokaryotes_values.loc[i,'plateau'], 0.1)))
prokaryotes_lysate = prokaryotes_lysate.merge(prokaryotes_values, on= 'Protein_ID', how = 'left')

Filtering of data frame for essential proteins with +1.5° C and -1.5° C of the maximal growth temperature (Tmax) around the T90 value. The following Tmax values for each organism were obtained from literature:

Bacillus subtilis: 55°C​<br/>
Escherichia coli: 45°C<br/>
Oleispira antarctica: 28°C<br/>
Picrophilus torridus: 66°C<br/>
Thermus thermophilus: 82°C<br/>

In [4]:
# Conditions for filtering the DataFrame
conditions = [
    (prokaryotes_lysate['run_name'] == 'Bacillus subtilis_168_lysate_R1') & (prokaryotes_lysate['T90'].between(53.5, 56.5)),
    (prokaryotes_lysate['run_name'] == 'Thermus thermophilus HB27 lysate') & (prokaryotes_lysate['T90'].between(80.5, 83.5)),
    (prokaryotes_lysate['run_name'] == 'Escherichia coli lysate') & (prokaryotes_lysate['T90'].between(43.5, 46.5)),
    (prokaryotes_lysate['run_name'] == 'Picrophilus torridus DSM9790 lysate') & (prokaryotes_lysate['T90'].between(64.5, 67.5)),
    (prokaryotes_lysate['run_name'] == 'Oleispira antarctica_RB-8_lysate_R1') & (prokaryotes_lysate['T90'].between(26.5, 29.5))
]
# Combine all conditions using bitwise OR
combined_condition = conditions[0]
for condition in conditions[1:]:
    combined_condition |= condition
# Filter the DataFrame based on the combined condition

essential_proteins = prokaryotes_lysate[combined_condition].reset_index(drop=True)
essential_proteins = essential_proteins.drop_duplicates(subset='Protein_ID', keep='first')
essential_proteins = essential_proteins[['Protein_ID', 'Organism', 'Protein names', 'meltPoint_x', 'AlphaFoldDB', 'PDB', 'Sequence', 'solution', 'Gene Ontology (biological process)']]
essential_proteins['Gene Ontology (biological process)'] = essential_proteins['Gene Ontology (biological process)'].str.split(';').str[0]
essential_proteins.to_csv(os.path.join(path, 'essential_proteins.csv'), index=False)

Grouping of all essential proteins based on the 5 groups aquired through checking common linked words in the GO category of biological processes:
- Protein Transport/Folding 
- Metabolism
- Transcription/Translation
- DNA Replication/Repair
- Cell division/cycle

In [44]:
for i in essential_proteins.index:
    essential_proteins['Gene Ontology (biological process)'] = essential_proteins['Gene Ontology (biological process)'].fillna('Unknown')
    if any(word in essential_proteins.loc[i, 'Gene Ontology (biological process)'] for word in ['transport', 'localization', 'protein', 'chaperone', 'assembly']):
        essential_proteins.loc[i, 'Gene Ontology (biological process)_obscure'] = 'Protein Transport/Folding'
    elif any(word in essential_proteins.loc[i, 'Gene Ontology (biological process)'] for word in ['metabolism', 'biosynthesis', 'catabolism', 'metabolic', 'anabolism', 'glycolysis', 'glycolytic', 'glucose', 'catabolic', 'biosynthetic', 'synthesis', 'carbon', 'fixation', 'respiration', 'salvage', 'gluconeogenesis', 'beta-oxidation']):
        essential_proteins.loc[i, 'Gene Ontology (biological process)_obscure'] = 'Metabolism'
    elif any(word in essential_proteins.loc[i, 'Gene Ontology (biological process)'] for word in ['RNA', 'transcription', 'translation', 'ribosome', 'mRNA', 'tRNA', 'rRNA', 'snoRNA', 'snRNA', 'siRNA', 'piRNA']):
        essential_proteins.loc[i, 'Gene Ontology (biological process)_obscure'] = 'Transcription/Translation'
    elif any(word in essential_proteins.loc[i, 'Gene Ontology (biological process)'] for word in ['replication', 'repair', 'damage', 'nucleotide', 'methylation']):
        essential_proteins.loc[i, 'Gene Ontology (biological process)_obscure'] = 'DNA Replication/Repair'
    elif any(word in essential_proteins.loc[i, 'Gene Ontology (biological process)'] for word in ['cell', 'division', 'cycle', 'growth', 'cellular', 'mitosis', 'meiosis', 'cytokinesis', 'chromosome', 'spindle', 'centrosome', 'centriole', 'centromere', 'kinetochore', 'telomere', 'nucleus', 'nuclear', 'nucleolus']):
        essential_proteins.loc[i, 'Gene Ontology (biological process)_obscure'] = 'Cell Division/Cycle'
    elif essential_proteins.loc[i, 'Gene Ontology (biological process)'] != 'Unknown':
        essential_proteins.loc[i, 'Gene Ontology (biological process)_obscure'] = 'Other'


essential_go = essential_proteins.loc[:, ['Protein names', 'Gene Ontology (biological process)_obscure', 'Gene Ontology (biological process)','Organism']]

Plotting a bar plot of counts of essential proteins for each GO category for E.coli and B.subtilis

In [ ]:
plt.figure(figsize=(12, 8))
sns.countplot(data=essential_proteins[essential_proteins['Organism'].isin(['Bacillus subtilis (strain 168)', 'Escherichia coli (strain K12)'])], x='Organism', hue='Gene Ontology (biological process)_obscure', palette='deep')
plt.xticks(ticks=plt.gca().get_xticks(), labels= ['B. subtilis', 'E. coli'])
plt.ylabel('Count of essential proteins')
plt.legend(title='Biological process')
plt.show()
plt.savefig('./data/essentialproteinsgraph.svg',format = 'svg', dpi =1000)


Calculating temperature dependent error of the regression for predicted melting points of mutated proteins. The respective function was taken from pca & regression jupyter notebook. The error of the WT prediction was calculated as its difference from the actual meltPoint value.

In [29]:
a = 7.456947114103443e-08
b = -3.19495740695305e-05
c = 0.0056160605705701215
d = -0.5174164345246695
e = 26.292297400105916
f = -697.0084579985947
g = 7524.223396560092
def error_func(temperature):
    return a*temperature**6 + b*temperature**5 + c*temperature**4 + d*temperature**3 + e*temperature**2 + f*temperature + g
mutated_proteins['error_Mutpred'] = mutated_proteins['Mut_prediction'].apply(lambda x: error_func(x))
mutated_proteins = mutated_proteins.dropna(subset=['Mut_prediction'])
mutated_proteins['error_WTpred'] = mutated_proteins['WT_prediction'] - mutated_proteins['meltPoint_x']

Computing wilcoxon signed rank test with integrating regression errors: This is done by bootstrapping each data point by adding a random error from a normal distribution of errors.

In [49]:
def wilcoxon_bootstrap(WT_pred, Mut_pred, error_WT, error_mut, num_bootstrap_samples):
    w_statistics = []
    p_values = []

    for _ in range(num_bootstrap_samples):
        # Generate a bootstrap samples with error noise generated from the regression errors
        bootstrap_sample_WT = WT_pred + np.random.normal(0, error_WT)
        bootstrap_sample_mut = Mut_pred + np.random.normal(0, error_mut)
        
        # Perform wilcoxon signed rank test on the bootstrap samples
        w_statistic, p_value = wilcoxon(bootstrap_sample_WT, bootstrap_sample_mut)
        w_statistics.append(w_statistic)
        p_values.append(p_value)

    mean_w_statistic = np.mean(w_statistics)
    mean_p_value = np.mean(p_values)
    return mean_w_statistic, mean_p_value

print(mutated_proteins.groupby('Organism').apply(
    lambda x: wilcoxon_bootstrap(x['WT_prediction'], x['Mut_prediction'], abs(x['error_WTpred']), abs(x['error_Mutpred']), 1000)
))

Organism
Bacillus subtilis (strain 168)                                                              (410.778, 3.60180353595941e-06)
Escherichia coli (strain K12)                                                               (98.058, 0.0009312822583015076)
Picrophilus torridus (strain ATCC 700027 / DSM 9790 / JCM 10055 / NBRC 100828 / KAW 2/3)                     (0.496, 0.748)
Thermus thermophilus (strain ATCC BAA-163 / DSM 7039 / HB27)                                                 (1.488, 0.622)
dtype: object


/var/folders/h6/x5s86vbj0n703jtjkk8l5q7r0000gp/T/ipykernel_2071/56312374.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  print(mutated_proteins.groupby('Organism').apply(
